In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_171"; Java(TM) SE Runtime Environment (build 1.8.0_171-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from /Users/raj/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/gl/wb7rqfgd3v708q5m0ydj3pkc0000gq/T/tmpj_0o9kqf
  JVM stdout: /var/folders/gl/wb7rqfgd3v708q5m0ydj3pkc0000gq/T/tmpj_0o9kqf/h2o_raj_started_from_python.out
  JVM stderr: /var/folders/gl/wb7rqfgd3v708q5m0ydj3pkc0000gq/T/tmpj_0o9kqf/h2o_raj_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.1
H2O cluster version age:,12 days
H2O cluster name:,H2O_from_python_raj_pgp0fp
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
url = "http://h2o-public-test-data.s3.amazonaws.com/smalldata/iris/iris_wheader.csv"
iris=h2o.import_file(url)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train,test= iris.split_frame([0.8])

In [5]:
train.nrows

123

In [6]:
test.nrows

27

In [8]:
from h2o.automl import H2OAutoML

Run time could be any default or may be hour or so

In [10]:
mA=H2OAutoML(max_runtime_secs=30)
mA.train(['sepal_len','sepal_wid','petal_len','petal_wid'],"class",train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
mA

In [14]:
mA.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
DRF_0_AutoML_20180619_141403,0.0111111,0.0613333,0.137673,0.018954
GBM_grid_0_AutoML_20180619_141403_model_15,0.0111111,0.116982,0.148999,0.0222008
GBM_grid_0_AutoML_20180619_141403_model_5,0.0114943,0.0354517,0.107989,0.0116616
XRT_0_AutoML_20180619_141403,0.0222222,0.0609717,0.130885,0.0171309
GBM_grid_0_AutoML_20180619_141403_model_16,0.0226054,0.0426859,0.124119,0.0154055
GBM_grid_0_AutoML_20180619_141403_model_0,0.0226054,0.0459365,0.117364,0.0137744
GBM_grid_0_AutoML_20180619_141403_model_1,0.0226054,0.020132,0.0871536,0.00759576
GBM_grid_0_AutoML_20180619_141403_model_14,0.0226054,0.108208,0.146754,0.0215366
GBM_grid_0_AutoML_20180619_141403_model_3,0.0226054,0.0313104,0.108281,0.0117247
GLM_grid_0_AutoML_20180619_141403_model_0,0.0226054,0.0708154,0.142862,0.0204097


To use the best model, call the leader

In [12]:
p=mA.leader.predict(test)

drf prediction progress: |████████████████████████████████████████████████| 100%


In [15]:
p

predict,Iris-setosa,Iris-versicolor,Iris-virginica
Iris-setosa,1,0,0
Iris-setosa,1,0,0
Iris-setosa,1,0,0
Iris-setosa,1,0,0
Iris-setosa,1,0,0
Iris-setosa,1,0,0
Iris-versicolor,0.00286533,0.830946,0.166189
Iris-versicolor,0.00215054,0.997849,0
Iris-versicolor,0.00215054,0.997849,0
Iris-versicolor,0.105085,0.894915,0


In [16]:
mA.leader.model_performance(test)


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.06683971210086596
RMSE: 0.258533773617425
LogLoss: 1.3787501455421678
Mean Per-Class Error: 0.05128205128205129
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
6.0,0.0,0.0,0.0,0 / 6
0.0,8.0,0.0,0.0,0 / 8
0.0,2.0,11.0,0.1538462,2 / 13
6.0,10.0,11.0,0.0740741,2 / 27


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9259259
2,0.9629629
3,0.9999999


leadeboard also uses 15% of your training data for your validation, so we can take a manual control on how we split our data